# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [107]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals: 
You have a target of selling 5500 units. 
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target. 
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours: 
You have both full-time and part-time employees. 
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours. 
Similarly, like sales, there could be deviations in these working hours. 

Overtime: 
Additionally, your employees might end up working overtime. 
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you. 
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities. 
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Ask for parameters

In [139]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [140]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**Sales:**

*  `S`:  Total units sold 
*  `Sd`: Deviation from the sales target (can be positive or negative)

**Employee Hours:**

*  `FTh`: Total hours worked by full-time employees
*  `FThd`: Deviation in full-time employee hours (can be positive or negative)
*  `PTh`: Total hours worked by part-time employees
*  `PThd`: Deviation in part-time employee hours (can be positive or negative)

**Overtime:**

*  `FTo`: Total overtime hours for full-time employees
*  `FTod`: Deviation from the overtime target for full-time employees (can be positive or negative)
*  `PTo`: Total overtime hours for part-time employees 


# 2. Ask for objective

In [141]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [142]:
Markdown(response2.text)

```
Minimize: 

P1 * |Sd| + P2 * |FTod| + 2 * P3 * |FThd| + P3 * |PThd| + P4 * |FTo| + 3 * P4 * |PTo| 
``` 


# 3. Ask for constraints

In [143]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [144]:
Markdown(response3.text)

## Constraints:

**Sales:**

*  `S = 5500 + Sd`  
*  `S = 5 * FTh + 2 * PTh` 

**Employee Hours:**

*  `FTh = 800 + FThd` 
*  `PTh = 320 + PThd` 

**Overtime:**

*  `FTo = 100 + FTod` 
*  `FTo >= 0`
*  `PTo >= 0`

**Variable Domains:**

* All deviations variables (`Sd`, `FThd`, `PThd`, `FTod`) can be positive, negative or zero.
* All other variables (`S`, `FTh`, `PTh`, `FTo`, `PTo`) should be non-negative. 


## 3. Generate the pyomo code

In [145]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [146]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if needed
P1 = 1  
P2 = 2 
P3 = 1.5 
P4 = 3  

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(domain=pyo.NonNegativeReals)
model.Sd = pyo.Var(domain=pyo.Reals)
model.FTh = pyo.Var(domain=pyo.NonNegativeReals)
model.FThd = pyo.Var(domain=pyo.Reals)
model.PTh = pyo.Var(domain=pyo.NonNegativeReals)
model.PThd = pyo.Var(domain=pyo.Reals)
model.FTo = pyo.Var(domain=pyo.NonNegativeReals)
model.FTod = pyo.Var(domain=pyo.Reals)
model.PTo = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
model.obj = pyo.Objective(
    expr= P1 * abs(model.Sd) 
           + P2 * abs(model.FTod) 
           + 2 * P3 * abs(model.FThd) 
           + P3 * abs(model.PThd) 
           + P4 * abs(model.FTo) 
           + 3 * P4 * abs(model.PTo),
    sense=pyo.minimize
)

# Constraints
model.sales_constraint = pyo.Constraint(expr=model.S == 5500 + model.Sd)
model.production_constraint = pyo.Constraint(expr=model.S == 5 * model.FTh + 2 * model.PTh)

model.fthours_constraint = pyo.Constraint(expr=model.FTh == 800 + model.FThd)
model.pthours_constraint = pyo.Constraint(expr=model.PTh == 320 + model.PThd)

model.ftovertime_constraint = pyo.Constraint(expr=model.FTo == 100 + model.FTod)
model.ftovertime_limit = pyo.Constraint(expr=model.FTo >= 0)
model.ptovertime_limit = pyo.Constraint(expr=model.PTo >= 0)

# Solve
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Sales (S):", pyo.value(model.S))
print("Full-time Hours (FTh):", pyo.value(model.FTh))
print("Part-time Hours (PTh):", pyo.value(model.PTh))
print("Full-time Overtime (FTo):", pyo.value(model.FTo))
print("Part-time Overtime (PTo):", pyo.value(model.PTo))
```

**Explanation and Sample Data:**

1. **Import Pyomo:** Import the Pyomo library.
2. **Sample Data:** Define the values of `P1`, `P2`, `P3`, and `P4`.  These represent the weights in your objective function. **You'll need to replace these with your actual data.**
3. **Model:** Create a concrete model object.
4. **Variables:** Define all your variables with their domains:
   -  `domain=pyo.NonNegativeReals` ensures the variable is greater than or equal to zero.
   -  `domain=pyo.Reals` allows the variable to be positive, negative, or zero.
5. **Objective Function (`model.obj`):**
   -  This implements your minimization objective using the `abs()` function for absolute values.
6. **Constraints:**  Each constraint is defined using `pyo.Constraint`.
7. **Solver:**  A solver is chosen (`cbc` in this example, but you might use others like 'glpk', 'gurobi', etc.).
8. **Solve:** The model is solved, and the results are stored.
9. **Print Results:** The script prints the solver status and the optimal values for the variables.

Remember to install Pyomo and a suitable solver before running this code. You can often install them using `pip`:

```bash
pip install pyomo
pip install cbc 
```

Replace the sample data with your real values and run the code to find the optimal solution. 


## 4. Input problem data and try running the generated code

In [155]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data if needed
P1 = 4  
P2 = 3 
P3 = 2
P4 = 1 

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(domain=pyo.NonNegativeReals)
model.Sd = pyo.Var(domain=pyo.Reals)
model.FTh = pyo.Var(domain=pyo.NonNegativeReals)
model.FThd = pyo.Var(domain=pyo.Reals)
model.PTh = pyo.Var(domain=pyo.NonNegativeReals)
model.PThd = pyo.Var(domain=pyo.Reals)
model.FTo = pyo.Var(domain=pyo.NonNegativeReals)
model.FTod = pyo.Var(domain=pyo.Reals)
model.PTo = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
model.obj = pyo.Objective(
    expr= P1 * abs(model.Sd) 
           + P2 * abs(model.FTod) 
           + 2 * P3 * abs(model.FThd) 
           + P3 * abs(model.PThd) 
           + P4 * abs(model.FTo) 
           + 3 * P4 * abs(model.PTo),
    sense=pyo.minimize
)

# Constraints
model.sales_constraint = pyo.Constraint(expr=model.S == 5500 + model.Sd)
model.production_constraint = pyo.Constraint(expr=model.S == 5 * model.FTh + 2 * model.PTh)

model.fthours_constraint = pyo.Constraint(expr=model.FTh == 800 + model.FThd)
model.pthours_constraint = pyo.Constraint(expr=model.PTh == 320 + model.PThd)

model.ftovertime_constraint = pyo.Constraint(expr=model.FTo == 100 + model.FTod)
model.ftovertime_limit = pyo.Constraint(expr=model.FTo >= 0)
model.ptovertime_limit = pyo.Constraint(expr=model.PTo >= 0)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Sales (S):", pyo.value(model.S))
print("Full-time Hours (FTh):", pyo.value(model.FTh))
print("Part-time Hours (PTh):", pyo.value(model.PTh))
print("Full-time Overtime (FTo):", pyo.value(model.FTo))
print("Part-time Overtime (PTo):", pyo.value(model.PTo))

ValueError: Model objective (obj) contains nonlinear terms that cannot be written to LP format

In [150]:
#This was run in collab using couenne so it can be run in this non-linear form. Here is the output:


# Problem: 
# - Lower bound: -inf
#   Upper bound: inf
#   Number of objectives: 1
#   Number of constraints: 0
#   Number of variables: 9
#   Sense: unknown
# Solver: 
# - Status: ok
#   Message: couenne\x3a Optimal
#   Termination condition: optimal
#   Id: 3
#   Error rc: 0
#   Time: 0.18695688247680664
# Solution: 
# - number of solutions: 0
#   number of solutions displayed: 0

# Sales (S): 5500.000000000001
# Full-time Hours (FTh): 972.0000000000001
# Part-time Hours (PTh): 319.99999999999994
# Full-time Overtime (FTo): 100.0
# Part-time Overtime (PTo): 0.0

## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [151]:
print(response.text)

## Variables:

**Sales:**

*  `S`:  Total units sold 
*  `Sd`: Deviation from the sales target (can be positive or negative)

**Employee Hours:**

*  `FTh`: Total hours worked by full-time employees
*  `FThd`: Deviation in full-time employee hours (can be positive or negative)
*  `PTh`: Total hours worked by part-time employees
*  `PThd`: Deviation in part-time employee hours (can be positive or negative)

**Overtime:**

*  `FTo`: Total overtime hours for full-time employees
*  `FTod`: Deviation from the overtime target for full-time employees (can be positive or negative)
*  `PTo`: Total overtime hours for part-time employees 



In [152]:
print(response2.text)

```
Minimize: 

P1 * |Sd| + P2 * |FTod| + 2 * P3 * |FThd| + P3 * |PThd| + P4 * |FTo| + 3 * P4 * |PTo| 
``` 



In [153]:
print(response3.text)

## Constraints:

**Sales:**

*  `S = 5500 + Sd`  
*  `S = 5 * FTh + 2 * PTh` 

**Employee Hours:**

*  `FTh = 800 + FThd` 
*  `PTh = 320 + PThd` 

**Overtime:**

*  `FTo = 100 + FTod` 
*  `FTo >= 0`
*  `PTo >= 0`

**Variable Domains:**

* All deviations variables (`Sd`, `FThd`, `PThd`, `FTod`) can be positive, negative or zero.
* All other variables (`S`, `FTh`, `PTh`, `FTo`, `PTo`) should be non-negative. 



In [154]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if needed
P1 = 1  
P2 = 2 
P3 = 1.5 
P4 = 3  

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(domain=pyo.NonNegativeReals)
model.Sd = pyo.Var(domain=pyo.Reals)
model.FTh = pyo.Var(domain=pyo.NonNegativeReals)
model.FThd = pyo.Var(domain=pyo.Reals)
model.PTh = pyo.Var(domain=pyo.NonNegativeReals)
model.PThd = pyo.Var(domain=pyo.Reals)
model.FTo = pyo.Var(domain=pyo.NonNegativeReals)
model.FTod = pyo.Var(domain=pyo.Reals)
model.PTo = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
model.obj = pyo.Objective(
    expr= P1 * abs(model.Sd) 
           + P2 * abs(model.FTod) 
           + 2 * P3 * abs(model.FThd) 
           + P3 * abs(model.PThd) 
           + P4 * abs(model.FTo) 
           + 3 * P4 * abs(model.PTo),
    sense=pyo.minimize
)

# Constraints
model.sales_constraint = pyo.Constraint(expr=model.S == 5500 + model.Sd)
model.production_constraint = pyo.Constraint(expr=mode